In [63]:
from torch.utils.data import Dataset
from torch import nn
import torch
from torch.utils.data import DataLoader
#from q71 import SLPNet
from q73 import NewsDataset
from q74 import load_Dataloader
import time
from torch.nn import functional as F

class SLPNet(nn.Module):
    def __init__(self, input_size, output_size,mid_size):        
        super().__init__()
        self.fc = nn.Linear(input_size,mid_size,bias=False)        
        self.fc2= nn.Linear(mid_size, mid_size)
        self.fc3=nn.Linear(mid_size, output_size)
        #正則化とドロップアウトを追加
        self.dropout = nn.Dropout(0.3)
        self.bn = nn.BatchNorm1d(mid_size)
        nn.init.normal_(self.fc.weight, 0.0, 1.0)  # 正規乱数で重みを初期化
        
    def forward(self, x):
        x = F.relu(self.fc(x))
        #x = F.relu(self.fc2(x))
        #x = self.dropout(x)
        x = F.relu(self.bn(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))        
#        x = self.relu(x)
#        x = self.dropout(x)
        
        return x


def calculate_loss_and_accuracy(model, criterion, loader, device):
    model.eval()
    loss = 0.0
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss += criterion(outputs, labels).item()
            pred = torch.argmax(outputs, dim=-1)
            total += len(inputs)
            correct += (pred == labels).sum().item()
            
    return loss / len(loader), correct / total


def train_model(dataloader_train, dataloader_valid,model, criterion, optimizer, num_epochs, device=None):
    # GPUに送る
    model.to(device)

    
    # 学習
    log_train = []
    log_valid = []
    
    scheduler =torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs, eta_min=1e-5, last_epoch=-1)
    #cosカーブに従って、学習率を変化させる
    #引数:オプティマイザ、半周期の長さ、最小学習率
    
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer,num_epochs/5,0.5)
    for epoch in range(num_epochs):
        # 開始時刻の記録
        s_time = time.time()
        
        # 訓練モードに設定 
        model.train()
        
        for inputs, labels in dataloader_train:
            # 勾配をゼロで初期化
            optimizer.zero_grad()
            # 順伝播 + 誤差逆伝播 + 重み更新
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
        # 損失と正解率の算出
        loss_train, acc_train = calculate_loss_and_accuracy(model, criterion, dataloader_train, device)
        loss_valid, acc_valid = calculate_loss_and_accuracy(model, criterion, dataloader_valid, device)
        log_train.append([loss_train, acc_train])
        log_valid.append([loss_valid, acc_valid])
            
        # チェックポイントの保存
#        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, f'checkpoint{epoch + 1}.pt')
            
        # 終了時刻の記録
        e_time = time.time()
        
        # ログを出力
        print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}, {(e_time - s_time):.4f}sec') 
        
#        if epoch > 2 and log_valid[epoch - 4][0] <= log_valid[epoch - 3][0] <= log_valid[epoch - 2][0] <= log_valid[epoch - 1][0] <= log_valid[epoch][0]:
#            break
        scheduler.step()
    return {'train': log_train, 'valid': log_valid}

if __name__=="__main__":

    
    model = SLPNet(300, 4,200)# モデルの定義    
    criterion = nn.CrossEntropyLoss()# 損失関数の定義
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)# オプティマイザの定義
    #optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    device = torch.device('cuda')# デバイスの指定
    
    # モデルの学習
    dataloader_train = load_Dataloader("train",64)
    dataloader_valid = load_Dataloader("valid", 64)
    dataloader_test = load_Dataloader("test", 64)
    log = train_model(dataloader_train, dataloader_valid,model, criterion, optimizer, 100,device=device)
    
    loss_train, acc_train = calculate_loss_and_accuracy(model, criterion, dataloader_train, device)
    loss_test, acc_test = calculate_loss_and_accuracy(model, criterion, dataloader_test, device)
    print(f"学習データでの正解率:{acc_train}")
    print(f"評価データでの正解率:{acc_test}")

epoch: 1, loss_train: 0.5758, accuracy_train: 0.8032, loss_valid: 0.5701, accuracy_valid: 0.8066, 0.4184sec
epoch: 2, loss_train: 0.4805, accuracy_train: 0.8233, loss_valid: 0.4805, accuracy_valid: 0.8216, 0.4030sec
epoch: 3, loss_train: 0.4382, accuracy_train: 0.8324, loss_valid: 0.4477, accuracy_valid: 0.8291, 0.4289sec
epoch: 4, loss_train: 0.4122, accuracy_train: 0.8399, loss_valid: 0.4318, accuracy_valid: 0.8313, 0.4089sec
epoch: 5, loss_train: 0.3927, accuracy_train: 0.8483, loss_valid: 0.4221, accuracy_valid: 0.8388, 0.3951sec
epoch: 6, loss_train: 0.3167, accuracy_train: 0.8934, loss_valid: 0.3504, accuracy_valid: 0.8831, 0.3858sec
epoch: 7, loss_train: 0.2967, accuracy_train: 0.9013, loss_valid: 0.3377, accuracy_valid: 0.8906, 0.3979sec
epoch: 8, loss_train: 0.2820, accuracy_train: 0.9050, loss_valid: 0.3360, accuracy_valid: 0.8921, 0.3930sec
epoch: 9, loss_train: 0.2677, accuracy_train: 0.9132, loss_valid: 0.3269, accuracy_valid: 0.8898, 0.3860sec
epoch: 10, loss_train: 0.256

epoch: 77, loss_train: 0.0411, accuracy_train: 0.9900, loss_valid: 0.3823, accuracy_valid: 0.9003, 0.3994sec
epoch: 78, loss_train: 0.0411, accuracy_train: 0.9905, loss_valid: 0.3755, accuracy_valid: 0.9003, 0.4049sec
epoch: 79, loss_train: 0.0414, accuracy_train: 0.9900, loss_valid: 0.3817, accuracy_valid: 0.9010, 0.3939sec
epoch: 80, loss_train: 0.0411, accuracy_train: 0.9900, loss_valid: 0.3787, accuracy_valid: 0.9003, 0.3920sec
epoch: 81, loss_train: 0.0406, accuracy_train: 0.9903, loss_valid: 0.3755, accuracy_valid: 0.9040, 0.4099sec
epoch: 82, loss_train: 0.0402, accuracy_train: 0.9903, loss_valid: 0.3760, accuracy_valid: 0.9048, 0.4266sec
epoch: 83, loss_train: 0.0399, accuracy_train: 0.9903, loss_valid: 0.3798, accuracy_valid: 0.9018, 0.3893sec
epoch: 84, loss_train: 0.0416, accuracy_train: 0.9901, loss_valid: 0.3834, accuracy_valid: 0.8996, 0.3990sec
epoch: 85, loss_train: 0.0390, accuracy_train: 0.9906, loss_valid: 0.3817, accuracy_valid: 0.9010, 0.4269sec
epoch: 86, loss_tra